In [1]:
import gc
import os
import pickle
import random
import shutil

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import losses, optimizers, metrics, callbacks

In [2]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [3]:
SEED = 123
N_CLASS = 12
MAX_EPOCHS = 200

In [4]:
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Load data

In [5]:
train_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/train",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=True,
    seed=SEED
)

val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/val",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=False
)

test_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/test",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=False
)

label_names = np.array(train_ds.class_names)
print("label names:", label_names)

Found 45586 files belonging to 12 classes.
Found 6513 files belonging to 12 classes.
Found 13024 files belonging to 12 classes.
label names: ['down' 'go' 'left' 'no' 'off' 'on' 'right' 'silence' 'stop' 'unknown'
 'up' 'yes']


In [6]:
def squeeze(audio, labels):
    audio = tf.squeeze(audio, axis=-1)
    return audio, labels

train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)
test_ds = test_ds.map(squeeze, tf.data.AUTOTUNE)

## CNN + LSTM Simple

In [7]:
from SpeechModels import get_melspec_model
import tensorflow.keras.layers as L
from keras.models import Model

def CNN_LSTM(n_out, input_length, kernel_size):

    mel_spec_model = get_melspec_model(input_length)
    inputs, outputs = mel_spec_model.inputs, mel_spec_model.outputs
    o_shape = outputs[0].shape
    y1 = L.Conv1D(32, kernel_size, activation='relu', padding='same', input_shape=o_shape[1:])(outputs[0])

    x = L.Bidirectional(L.LSTM(128, return_sequences=False))(y1)  # [b_s, seq_len, vec_dim]

    output = L.Dense(n_out, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model


model = CNN_LSTM(N_CLASS, 16000, 5)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 16000)]      0           []                               
                                                                                                  
 tf.signal.stft (TFOpLambda)    (None, 125, 513)     0           ['input[0][0]']                  
                                                                                                  
 tf.math.abs (TFOpLambda)       (None, 125, 513)     0           ['tf.signal.stft[0][0]']         
                                                                                                  
 tf.tensordot (TFOpLambda)      (None, 125, 80)      0           ['tf.math.abs[0][0]']            
                                                                                              

In [8]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=losses.SparseCategoricalCrossentropy(),
    metrics=[metrics.SparseCategoricalAccuracy(), metrics.SparseCategoricalCrossentropy()]
)
checkpoint = callbacks.ModelCheckpoint(
    filepath="simple_cnn.h5",
    monitor="val_sparse_categorical_accuracy",
    verbose=0,
    save_best_only=True,
    save_weights_only=True
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_sparse_categorical_accuracy',
    min_delta=0,
    patience=5,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_sparse_categorical_accuracy', factor=0.5, patience=3, min_lr=0.00001, verbose=1)

In [9]:
history = model.fit(
    train_ds,
    epochs=2,
    validation_data=val_ds,
    shuffle=True,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

Epoch 1/2
90/90 [==============================] - 22s 115ms/step - loss: 1.4897 - sparse_categorical_accuracy: 0.6263 - sparse_categorical_crossentropy: 1.4897 - val_loss: 1.3607 - val_sparse_categorical_accuracy: 0.6329 - val_sparse_categorical_crossentropy: 1.3607 - lr: 0.0010
Epoch 2/2
90/90 [==============================] - 13s 129ms/step - loss: 1.1501 - sparse_categorical_accuracy: 0.6571 - sparse_categorical_crossentropy: 1.1501 - val_loss: 1.0418 - val_sparse_categorical_accuracy: 0.6664 - val_sparse_categorical_crossentropy: 1.0418 - lr: 0.0010


## Experiments

Training will be repeated 5 times with different weights initialization.

In [10]:
EXPERIMENT_NAME = "cnn_lstm_simple"
if os.path.exists(EXPERIMENT_NAME):
    shutil.rmtree(EXPERIMENT_NAME)
    os.mkdir(EXPERIMENT_NAME)
else:
    os.mkdir(EXPERIMENT_NAME)

In [11]:
TRAINING_SEEDS = list(range(5))
results = []
for seed in TRAINING_SEEDS:
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    model = CNN_LSTM(N_CLASS, 16000, 5)


    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.01),
        loss=losses.SparseCategoricalCrossentropy(),
        metrics=[metrics.SparseCategoricalAccuracy(), metrics.SparseCategoricalCrossentropy()]
    )

    history = model.fit(
        train_ds,
        epochs=MAX_EPOCHS,
        validation_data=val_ds,
        shuffle=True,
        callbacks=[early_stopping, reduce_lr]
    )

    with open(os.path.join(EXPERIMENT_NAME, f"history_{seed}.pkl"), "wb") as file:
        pickle.dump(history.history, file)

    eval_results = model.evaluate(test_ds)

    predictions = model.predict(test_ds)
    with open(os.path.join(EXPERIMENT_NAME, f"predictions_{seed}.pkl"), "wb") as file:
        pickle.dump(predictions, file)

    results += [{
        'seed': seed,
        'results': dict(zip(model.metrics_names, eval_results))
    }]
    gc.collect()

results = pd.DataFrame(results)
results = pd.concat([results.drop(["results"], axis=1), results["results"].apply(pd.Series)], axis=1)
results.to_csv(os.path.join(EXPERIMENT_NAME, 'results.csv'))

Epoch 1/200
90/90 [==============================] - 14s 117ms/step - loss: 1.4694 - sparse_categorical_accuracy: 0.6248 - sparse_categorical_crossentropy: 1.4694 - val_loss: 1.3282 - val_sparse_categorical_accuracy: 0.6349 - val_sparse_categorical_crossentropy: 1.3282 - lr: 0.0100
Epoch 2/200
90/90 [==============================] - 11s 109ms/step - loss: 1.1510 - sparse_categorical_accuracy: 0.6630 - sparse_categorical_crossentropy: 1.1510 - val_loss: 0.9581 - val_sparse_categorical_accuracy: 0.6969 - val_sparse_categorical_crossentropy: 0.9581 - lr: 0.0100
Epoch 3/200
90/90 [==============================] - 11s 108ms/step - loss: 0.8099 - sparse_categorical_accuracy: 0.7415 - sparse_categorical_crossentropy: 0.8099 - val_loss: 0.7036 - val_sparse_categorical_accuracy: 0.7695 - val_sparse_categorical_crossentropy: 0.7036 - lr: 0.0100
Epoch 4/200
90/90 [==============================] - 10s 107ms/step - loss: 0.6383 - sparse_categorical_accuracy: 0.8007 - sparse_categorical_crossentr

In [12]:
results

,seed,loss,sparse_categorical_accuracy,sparse_categorical_crossentropy
0,0,0.202165,0.942107,0.202165
1,1,0.202767,0.942644,0.202767
2,2,0.184794,0.945025,0.184794
3,3,0.287714,0.915464,0.287714
4,4,0.185222,0.946560,0.185222
